**Author: [Carl McBride Ellis](https://www.kaggle.com/carlmcbrideellis)** ([LinkedIn](https://www.linkedin.com/in/carl-mcbride-ellis/))

## The Titanic without Jack or Rose: data pseudonymization using Faker
In this simple notebook we shall replace the real names of each of the Titanic passengers with a [pseudonym](https://en.wikipedia.org/wiki/Pseudonym). Such a process is known as pseudonymization which, in European Union law, is defined as:

> "*...the processing of personal data in such a manner that the personal data can no longer be attributed to a specific data subject without the use of additional information, provided that such additional information is kept separately and is subject to technical and organisational measures to ensure that the personal data are not attributed to an identified or identifiable natural person.*" [(GDPR) EU 2016/679 Article 4 (5)](https://eur-lex.europa.eu/eli/reg/2016/679)

These new names are generated randomly, and each time one runs the script the names will change. To do this we shall use the [Faker](https://pypi.org/project/Faker/) package. Let us load in the data and take a look at it:

In [1]:
!pip install -q Faker

ERROR: bokeh 2.1.1 has requirement tornado>=5.1, but you'll have tornado 5.0.2 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd 
# read in the Titanic training data csv file
train_data = pd.read_csv('../input/titanic/train.csv')
# take a look
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


we can see that the **Name** column contains [personally identifiable information](https://en.wikipedia.org/wiki/Personal_data) i.e. in this case the real names of the passengers. We shall replace that column with fake names using **Faker**. We shall use either male of female names based on the **Sex** column data.

In [3]:
from faker import Faker
fake = Faker()

def Sex(row):
    if row['Sex'] == 'female':
        new_name = fake.name_female()
    else:
        new_name = fake.name_male()
    return new_name

train_data['Name'] = train_data.apply(Sex, axis=1)

# take a quick look
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Johnny Bell,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Megan Thompson,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Lindsey Reed,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Courtney Martin,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,Jonathan White,male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,Matthew Wright II,male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,Cody Aguirre,male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,Jeffrey Mcdowell,male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,Samantha Larsen,female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,Cindy Parker,female,14.0,1,0,237736,30.0708,NaN,C


Finally we shall now write out this new pseudonymized DataFrame to a csv file:

In [4]:
train_data.to_csv('pseudonymized_train.csv', index=False)

## Hashing
Another perhaps less fanciful option is to create a [hash](https://en.wikipedia.org/wiki/Hash_function) of each passenger name, and to save the values to a dictionary to allow later de-anonymization by the dictionary owner:

In [5]:
import hashlib

train_data = pd.read_csv('../input/titanic/train.csv')

# create the hash
train_data['Name hash'] = train_data['Name'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest()[:8])

# save to a dictionary
name_lookup = dict(zip(train_data['Name hash'],train_data['Name']))

# now delete the "Name" column
train_data = train_data.drop(["Name"], axis=1)

# take a look
train_data.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name hash
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,6c969dc7
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,f7ad2d69
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,7eb1fa77
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,c8a06e74
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,885e53ea
5,6,0,3,male,NaN,0,0,330877,8.4583,NaN,Q,fab68f7e
6,7,0,1,male,54.0,0,0,17463,51.8625,E46,S,558f7a85
7,8,0,3,male,2.0,3,1,349909,21.0750,NaN,S,cbd46ee4
8,9,1,3,female,27.0,0,2,347742,11.1333,NaN,S,3f3b1d43
9,10,1,2,female,14.0,1,0,237736,30.0708,NaN,C,c54fde5d


The owner of the dictionary can de-code the values in the hash column, for example as follows:

In [6]:
print(name_lookup["6c969dc7"])

Braund, Mr. Owen Harris


Note that the new file is by no means rigorously anonymized as we have kept, for example, the **Ticket** column as it was, which could be used to de-anonymize the file. We also have **Cabin** information for some of the entries. Indeed, to truly anonymize data to legal standards is a far from trivial task. 

It is also worth stating that this notebook is just an exercise; the passenger list of the RMS Titanic available under Creative Commons, is a historical document, and is here being used *in the public interest, scientific or historical research purposes or statistical purposes* (Article 89).

## Links:
* [Faker (GitHub)](https://github.com/joke2k/faker)
* [Faker documentation](https://faker.readthedocs.io/en/stable/)

## Related reading:
* [Data anonymization](https://en.wikipedia.org/wiki/Data_anonymization)
* [Pseudonymization](https://en.wikipedia.org/wiki/Pseudonymization)
* [Data re-identification](https://en.wikipedia.org/wiki/Data_re-identification)
* [General Data Protection Regulation (GDPR)](https://eur-lex.europa.eu/eli/reg/2016/679) (EU) 2016/679 